ПЕРЕДЕЛАТЬ ДОК С ВИДАМИ ОРГАНОВ

In [ ]:

import os
from google.colab import drive
import zipfile
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import glob


drive.mount('/content/gdrive/')
organs = pd.read_csv('/content/gdrive/MyDrive/russia_organ_levels.csv')

!unzip -q /content/gdrive/My\ Drive/2000.zip -d train0
!unzip -q /content/gdrive/My\ Drive/2001.zip -d train1
!unzip -q /content/gdrive/My\ Drive/2002.zip -d train2
!unzip -q /content/gdrive/My\ Drive/2003.zip -d train3
!unzip -q /content/gdrive/My\ Drive/2004.zip -d train4
!unzip -q /content/gdrive/My\ Drive/2005.zip -d train5
!unzip -q /content/gdrive/My\ Drive/2006.zip -d train6
!unzip -q /content/gdrive/My\ Drive/2007.zip -d train7
!unzip -q /content/gdrive/My\ Drive/2008.zip -d train8
!unzip -q /content/gdrive/My\ Drive/2009.zip -d train9
!unzip -q /content/gdrive/My\ Drive/2010.zip -d train10
!unzip -q /content/gdrive/My\ Drive/2011.zip -d train11
!unzip -q /content/gdrive/My\ Drive/2012.zip -d train12
!unzip -q /content/gdrive/My\ Drive/2013.zip -d train13
!unzip -q /content/gdrive/My\ Drive/2014.zip -d train14
!unzip -q /content/gdrive/My\ Drive/2015-1.zip -d train15_1
!unzip -q /content/gdrive/My\ Drive/2015-2.zip -d train15_2
!unzip -q /content/gdrive/My\ Drive/2016-1.zip -d train16_1
!unzip -q /content/gdrive/My\ Drive/2016-2.zip -d train16_2
!unzip -q /content/gdrive/My\ Drive/2017-1.zip -d train17_1
!unzip -q /content/gdrive/My\ Drive/2017-2.zip -d train17_2
!unzip -q /content/gdrive/My\ Drive/2017-3.zip -d train17_3


KeyboardInterrupt: 

In [ ]:
organs = pd.read_csv('/content/gdrive/MyDrive/russia_organ_levels.csv')

In [ ]:
import os
from google.colab import drive
import zipfile
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import glob


drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


сначала загрузим документы из одного года, архив лежит на гугл-диске

In [ ]:
def get_df( year, file_dir):
  fieldsid = ['authorByOP']

  fieldshd = [ 'docTitleByOP', 'docTypeByOP']
  fieldstxt = ['textOCR',
               'textIPS']
  alldata = [] #we will collect dictionaries to this in order to make a Data Frame of it
  #iterate through XML files, parse a file into a dictionary and add to alldata
  for inputfilename in glob.glob(file_dir + str(year) + '*.xml'):
      tree = ET.parse(inputfilename)
      root = tree.getroot()
      bufferdict = {}
      bufferdict['fileid'] = inputfilename[-15:]
      for field in fieldsid:
          try:
            bufferdict[field] = root.find("./meta/identification/" + field).get('val')
          except:
            bufferdict[field] = None
      for field in fieldshd:
        try:
            bufferdict[field] = root.find("./meta/identification/" + field).text
        except:
            bufferdict[field] = None
      for field in fieldstxt:
        try:
            bufferdict[field] = root.find("./body/" + field).text
        except:
            bufferdict[field] = None
    #then since we can have multiple values from classifier and keywords field we have to represent lists of them via semicolon
      alldata.append(bufferdict)

# finally make a dataframe
  df = pd.DataFrame(alldata,  columns = ['fileid']+ fieldsid  + fieldshd + fieldstxt )
  df = df[df['authorByOP']!='n/f']
  df['doc_text'] = df['textOCR']
  df.loc[df['textIPS'].notna(), 'doc_text'] = df['textIPS']
  df = df[(df['doc_text'].str.len()>1000)&(df['doc_text'].str.len()<3000)]

  df_to_return = df[['docTypeByOP',  'doc_text','authorByOP']]
  df_to_return = df_to_return.dropna()

  return df_to_return

In [ ]:
organs = organs.rename(columns={'organ':'authorByOP'})

df_11 = get_df(2011, 'train11/')
df_12 = get_df(2012, 'train12/')
df_13 = get_df(2013, 'train13/')
df_14 = get_df(2014, 'train14/')
df_15_1 = get_df(2015, 'train15_1/')
df_15_2 = get_df(2015, 'train15_2/')
df_16_1 = get_df(2016, 'train16_1/')
df_16_2 = get_df(2016, 'train16_2/')
df_17_1 = get_df(2017, 'train17_1/')
df_17_2 = get_df(2017, 'train17_2/')
df_17_3 = get_df(2017, 'train17_3/')

In [ ]:
df = pd.concat([df_11, df_12,df_13, df_14,df_15_1, df_15_2,df_16_1, df_16_2,df_17_1, df_17_2, df_17_3])
df

,docTypeByOP,doc_text,authorByOP
7,Указ,\n УКАЗ ПР...,Президента Российской Федерации
86,Распоряжение,\n РАСПОРЯЖЕНИЕ ПР...,Президента Российской Федерации
154,Распоряжение,\n РАСПОРЯЖЕНИЕ ПР...,Президента Российской Федерации
210,Федеральный закон,\n РОССИЙСКАЯ ФЕДЕРАЦ...,Государственная Дума Федерального Собрания Рос...
301,Федеральный закон,\n РОССИЙСКАЯ ФЕДЕРАЦ...,Государственная Дума Федерального Собрания Рос...
...,...,...,...
21860,Указ,У К А З\n ...,Губернатора Пермского края
21861,Постановление,СМОЛЕНСКАЯ ОБЛАСТНАЯ ДУМА\n\n ...,Смоленской областной Думы
21865,Постановление,ПРАВИТЕЛЬСТВО КУРГАНСКОЙ ...,Департамента государственного регулирования це...
21878,Постановление,ЁЁЁ…ЁЁЁЁЁЁЁЁЁЁЁЁЁЁЁЁЁЁЁЁЁЁ ...,Правительства Республики Алтай


In [ ]:
df_table_1 = pd.DataFrame(df['docTypeByOP'].value_counts())
df_table_1 = df_table_1.reset_index()
df_table_1 = df_table_1.rename(columns={'index':'Вид нормативно-правового акта',
                                        'docTypeByOP': 'Количество вхождений документа'})
df_table_1.to_excel('table_1_doc_count_first.xlsx', index=False)

In [ ]:
df_level = df.merge(organs,on='authorByOP' )
df_level['docTypeByOP'] = df_level['docTypeByOP'].replace({'Федеральный закон':'Закон'})
df_level['doc_name'] = df_level['docTypeByOP'] + ', ' + df_level['art']

docs_types = ['Постановление, региональный уровень, правительство',
              'Закон, региональный уровень, законодательный орган',
              'Постановление, региональный уровень, глава исполнительной власти',
              'Указ, региональный уровень, глава исполнительной власти',
              'Постановление, региональный уровень, законодательный орган',
              'Распоряжение, федеральный уровень, правительство',
              'Приказ, региональный уровень, органы исполнительной власти',
              'Распоряжение, региональный уровень, правительство',
              'Распоряжение, региональный уровень, глава исполнительной власти',
              'Постановление, федеральный уровень, правительство',
              'Приказ, федеральный уровень, органы исполнительной власти',
              'Указ, федеральный уровень, глава исполнительной власти',
              'Постановление, региональный уровень, органы исполнительной власти',
              'Закон, федеральный уровень, законодательный орган',
              'Распоряжение, федеральный уровень, глава исполнительной власти'
              ]

df_passed = df_level[df_level['doc_name'].isin(docs_types)]
df_passed['doc_name'].value_counts()

Постановление, региональный уровень, правительство                   72003
Постановление, региональный уровень, глава исполнительной власти     13439
Закон, региональный уровень, законодательный орган                   12012
Указ, региональный уровень, глава исполнительной власти              10272
Приказ, региональный уровень, органы исполнительной власти            7119
Постановление, региональный уровень, законодательный орган            4992
Распоряжение, федеральный уровень, правительство                      3636
Распоряжение, региональный уровень, правительство                     3420
Приказ, федеральный уровень, органы исполнительной власти             2152
Распоряжение, региональный уровень, глава исполнительной власти       2088
Постановление, федеральный уровень, правительство                     2047
Указ, федеральный уровень, глава исполнительной власти                1216
Постановление, региональный уровень, органы исполнительной власти     1200
Закон, федеральный уровен

In [ ]:
df_table_2 = pd.DataFrame(df_level['doc_name'].value_counts())
df_table_2 = df_table_2.reset_index()
df_table_2 = df_table_2.rename(columns={'index':'Вид нормативно-правового акта',
                                        'doc_name': 'Количество вхождений документа'})
df_table_2.to_excel('table_2_doc_count_second.xlsx', index=False)

In [ ]:
def get_df_old( year, file_dir):
  fieldsid = ['IssuedByIPS']
  fieldstxt = [ 'textIPS']
  alldata = [] #we will collect dictionaries to this in order to make a Data Frame of it
  #iterate through XML files, parse a file into a dictionary and add to alldata
  for inputfilename in glob.glob(file_dir + str(year) + '*.xml'):
      tree = ET.parse(inputfilename)
      root = tree.getroot()
      bufferdict = {}
      bufferdict['fileid'] = inputfilename[-15:]
      for field in fieldsid:
          try:
            bufferdict[field] = root.find("./meta/identification/" + field).get('val')
          except:
            bufferdict[field] = None
      for field in fieldstxt:
        try:
            bufferdict[field] = root.find("./body/" + field).text
        except:
            bufferdict[field] = None
    #then since we can have multiple values from classifier and keywords field we have to represent lists of them via semicolon
      alldata.append(bufferdict)

# finally make a dataframe
  df = pd.DataFrame(alldata,  columns = ['fileid']+ fieldsid  + fieldstxt )
  df['doc_text'] = df['textIPS']
  df = df[(df['doc_text'].str.len()>1000)&(df['doc_text'].str.len()<3000)]
  df = df[(df['IssuedByIPS'].str.contains('Распоряжение Президента Российской Федерации'))|
           (df['IssuedByIPS'].str.contains('Федеральный закон'))]
  df_to_return = df[['doc_text','IssuedByIPS' ]]
  df_to_return = df_to_return.rename(columns={'IssuedByIPS':'doc_name'})
  df_to_return['doc_name'] = df_to_return['doc_name'].replace({'Распоряжение Президента Российской Федерации':'Распоряжение, федеральный уровень, глава исполнительной власти',
                         'Федеральный закон ':'Закон, федеральный уровень, законодательный орган'})
  df_to_return = df_to_return.dropna()

  return df_to_return

In [ ]:
df_0 = get_df_old(2000, 'train0/')
df_1 = get_df_old(2001, 'train1/')
df_2 = get_df_old(2002, 'train2/')
df_3 = get_df_old(2003, 'train3/')
df_4 = get_df_old(2004, 'train4/')
df_5 = get_df_old(2005, 'train5/')
df_6 = get_df_old(2006, 'train6/')
df_7 = get_df_old(2007, 'train7/')
df_8 = get_df_old(2008, 'train8/')
df_9 = get_df_old(2009, 'train9/')
df_10 = get_df_old(2010, 'train10/')
df_old = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9,df_10])
df_old

,doc_text,doc_name
24,\n РАСПОРЯЖЕНИЕ ПР...,"Распоряжение, федеральный уровень, глава испол..."
55,\n РАСПОРЯЖЕНИЕ ПР...,"Распоряжение, федеральный уровень, глава испол..."
137,\n РАСПОРЯЖЕНИЕ ПР...,"Распоряжение, федеральный уровень, глава испол..."
194,\n РОССИЙСКАЯ ФЕДЕРАЦ...,"Закон, федеральный уровень, законодательный орган"
220,\n РАСПОРЯЖЕНИЕ ПР...,"Распоряжение, федеральный уровень, глава испол..."
...,...,...
9095,\n РАСПОРЯЖЕНИЕ ПР...,"Распоряжение, федеральный уровень, глава испол..."
9105,\n РОССИЙСКАЯ ФЕДЕРАЦ...,"Закон, федеральный уровень, законодательный орган"
9122,\n РАСПОРЯЖЕНИЕ ПР...,"Распоряжение, федеральный уровень, глава испол..."
9145,\n РАСПОРЯЖЕНИЕ ПР...,"Распоряжение, федеральный уровень, глава испол..."


In [ ]:
df_zus = pd.concat([df_passed[['doc_name','doc_text' ]], df_old])
df_zus['doc_name'].value_counts()

Постановление, региональный уровень, правительство                   72003
Постановление, региональный уровень, глава исполнительной власти     13439
Закон, региональный уровень, законодательный орган                   12012
Указ, региональный уровень, глава исполнительной власти              10272
Приказ, региональный уровень, органы исполнительной власти            7119
Постановление, региональный уровень, законодательный орган            4992
Распоряжение, федеральный уровень, правительство                      3636
Распоряжение, региональный уровень, правительство                     3420
Приказ, федеральный уровень, органы исполнительной власти             2152
Распоряжение, региональный уровень, глава исполнительной власти       2088
Постановление, федеральный уровень, правительство                     2047
Закон, федеральный уровень, законодательный орган                     1956
Распоряжение, федеральный уровень, глава исполнительной власти        1275
Указ, федеральный уровень

In [ ]:
df_zus['doc_name'].unique().tolist()

['Указ, федеральный уровень, глава исполнительной власти',
 'Распоряжение, федеральный уровень, глава исполнительной власти',
 'Закон, федеральный уровень, законодательный орган',
 'Постановление, федеральный уровень, правительство',
 'Распоряжение, федеральный уровень, правительство',
 'Постановление, региональный уровень, законодательный орган',
 'Постановление, региональный уровень, правительство',
 'Распоряжение, региональный уровень, правительство',
 'Указ, региональный уровень, глава исполнительной власти',
 'Постановление, региональный уровень, глава исполнительной власти',
 'Распоряжение, региональный уровень, глава исполнительной власти',
 'Закон, региональный уровень, законодательный орган',
 'Приказ, федеральный уровень, органы исполнительной власти',
 'Приказ, региональный уровень, органы исполнительной власти',
 'Постановление, региональный уровень, органы исполнительной власти']

In [ ]:
need_sample = ['Закон, федеральный уровень, законодательный орган',
                'Распоряжение, федеральный уровень, правительство',
                'Постановление, федеральный уровень, правительство',
                'Постановление, региональный уровень, правительство',
                'Распоряжение, региональный уровень, правительство',
                'Постановление, региональный уровень, глава исполнительной власти',
                'Распоряжение, региональный уровень, глава исполнительной власти',
                'Указ, региональный уровень, глава исполнительной власти',
                'Постановление, региональный уровень, законодательный орган',
                'Закон, региональный уровень, законодательный орган',
                'Приказ, федеральный уровень, органы исполнительной власти',
                'Приказ, региональный уровень, органы исполнительной власти'
              ]

df_sample = df_zus[df_zus['doc_name'].isin(need_sample)]
sampled = df_sample.groupby('doc_name').sample(n=1600, random_state=42)
df_final = pd.concat([df_zus[~df_zus['doc_name'].isin(need_sample)], sampled])
df_final['doc_name'].value_counts()

Закон, региональный уровень, законодательный орган                   1600
Закон, федеральный уровень, законодательный орган                    1600
Постановление, региональный уровень, глава исполнительной власти     1600
Постановление, региональный уровень, законодательный орган           1600
Постановление, региональный уровень, правительство                   1600
Постановление, федеральный уровень, правительство                    1600
Приказ, региональный уровень, органы исполнительной власти           1600
Приказ, федеральный уровень, органы исполнительной власти            1600
Распоряжение, региональный уровень, глава исполнительной власти      1600
Распоряжение, региональный уровень, правительство                    1600
Распоряжение, федеральный уровень, правительство                     1600
Указ, региональный уровень, глава исполнительной власти              1600
Распоряжение, федеральный уровень, глава исполнительной власти       1275
Указ, федеральный уровень, глава испол

In [ ]:
df_table_3 = pd.DataFrame(df_final['doc_name'].value_counts())
df_table_3 = df_table_3.reset_index()
df_table_3 = df_table_3.rename(columns={'index':'Вид нормативно-правового акта',
                                        'doc_name': 'Количество вхождений документа'})
df_table_3.to_excel('table_3_doc_count_final.xlsx', index=False)


In [ ]:
df_final.to_csv('//content/gdrive/MyDrive/final_df_18_01.csv', index=False)

в тексте много опечаток и странных символов, идея - найти все эти символы во всех текстах документов и сохранить их в отдельный сет, вывести его и положить символы в функцию, которая почистит текст от них

функция для поиска всех плохих символов - то есть тех, которые не являются русскими буквами, пробелом или точкой

In [ ]:
df_final = pd.read_csv('//content/gdrive/MyDrive/final_df_17_01.csv')

In [ ]:
texts = df_final['doc_text'].tolist()

In [ ]:
def find_bad_symb(l1, l2):
  bad_symb = set()
  for i in range(len(l1)):
    bad_symb.update(set(l1[i])-set(l2))
  return bad_symb


russian_letters = [chr(i) for i in range(0x0410, 0x044F+1)]
russian_letters.append(' ')
russian_letters.append('.')
for i in range(0,11):
  russian_letters.append(str(i))

all_bed = set()
for i in texts:
  all_bed.update(find_bad_symb(i, russian_letters))
all_bed

{'\n',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '¦',
 '§',
 '©',
 '«',
 '®',
 '°',
 '»',
 '̀',
 'Ё',
 'ё',
 'і',
 '—',
 '‘',
 '’',
 '‚',
 '“',
 '”',
 '„',
 '…',
 '‹',
 '›',
 '№'}

теперь можно прямо перечислить все эти символы и положить их в функцию, которая очистит от них текст

In [ ]:
  u"\X"
  u"\g"
  u"\«"
  u"\©"
  u"\°"
  u"\'"
  u"\m"
  u"\#"
  u"\-"
  u"\z"
  u"\R"
  u"\^"
  u"\D"
  u"\,"
  u"\h"
  u"\‘"
  u"\…"
  u"\K"
  u"\»"
  u"\”"
  u"\M"
  u"\$"
  u"\—"
  u"\N"
  u"\Ё"
  u"\_"
  u"\B"
  u"\o"
  u"\)"
  u"\&"
  u"\>"
  u"\F"
  u"\q"
  u"\b"
  u"\f"
  u"\S"
  u"\]"
  u"\u"
  u"\d"
  u"\̀ "
  u"\|"
  u"\’"
  u"\@"
  u"\n"
  u"\;"
  u"\›"
  u"\\"
  u"\("
  u"\*"
  u"\№"
  u"\c"
  u"\y"
  u"\‚"
  u"\w"
  u"\P"
  u"\E"
  u"\„"
  u"\U"
  u"\V"
  u"\O"
  u"\l"
  u"\%"
  u"\і"
  u"\j"
  u"\="
  u"\p"
  u"\`"
  u"\“"
  u"\/"
  u"\C"
  u"\W"
  u"\v"
  u"\x"
  u"\T"
  u"\a"
  u"\""
  u"\["
  u"\+"
  u"\:"
  u"\Z"
  u"\Y"
  u"\§"
  u"\}"
  u"\L"
  u"\I"
  u"\k"
  u"\e"
  u"\!"
  u"\r"
  u"\A"
  u"\{"
  u"\‹"
  u"\i"
  u"\<"
  u"\ё"
  u"\t"
  u"\H"
  u"\®"
  u"\G"
  u"\s"
  u"\¦"
  u"\J"
  u"\?"

In [ ]:
import re

In [ ]:
def remove_bad_symbols(string):
  bad_symb = re.compile("["

                u"\№"
                u"\і"
                u"\ё"
                u"\{"
                u"\›"
                u"\`"
                u"\“"
                u"\№"
                u"\@"
                u"\—"
                u"\/"
                u"\°"
                u"\«"
                u"\,"
                u"\)"
                u"\…"
                u"\>"
                u"\="
                u"\#"
                u"\:"
                u"\і"
                u"\%"
                u"\]"
                u"\?"
                u"\‘"
                u"\_"
                u"\;"
                u"\["
                u"\*"
                u"\'"
                u"\""
                u"\$"
                u"\<"
                u"\^"
                u"\‚"
                u"\‹"
                u"\’"
                u"\”"
                u"\»"
                u"\&"
                u"\Ё"
                u"\("
                u"\„"
                u"\!"
                u"\|"
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
                 "]+", flags=re.UNICODE)
  return bad_symb.sub(r'', string)


data_test = df_final.copy()
data_test['doc_text'] = data_test['doc_text'].apply(remove_bad_symbols)
data_test['doc_text'] = data_test['doc_text'].str.replace('\n', ' ')
data_test['doc_text'] = data_test['doc_text'].str.replace('-', ' ')
data_test

,doc_name,doc_text
0,"Распоряжение, федеральный уровень, глава испол...",РАСПОРЯЖЕНИЕ ПРЕ...
1,"Распоряжение, федеральный уровень, глава испол...",РАСПОРЯЖЕНИЕ ПРЕ...
2,"Указ, федеральный уровень, глава исполнительно...",УКАЗ ПРЕ...
3,"Распоряжение, федеральный уровень, глава испол...",РАСПОРЯЖЕНИЕ ПРЕ...
4,"Распоряжение, федеральный уровень, глава испол...",РАСПОРЯЖЕНИЕ ПРЕ...
...,...,...
37358,"Указ, региональный уровень, глава исполнительн...",ГУБЕРНАТОР огловской ОБЛАСТИ У К АЗ ...
120850,"Указ, региональный уровень, глава исполнительн...",ммжхжчмитжкъщ. 3 а . . ...
40095,"Указ, региональный уровень, глава исполнительн...",У К А З ГУБЕРНАТОРА ПЕРМСКОГО КРАЯ 01.08.2...
40050,"Указ, региональный уровень, глава исполнительн...",У К А З ГУБЕРНАТОРА ПЕРМСКОГО КРАЯ 20.02.2...


In [ ]:
data_test.to_csv('//content/gdrive/MyDrive/final_df_18_01.csv', index=False)

в текстах все равно сохранились опечатки в некоторых словах. Для того, чтобы исправить хотя бы одну ошибку в слове, прогоним текст через модель от DeepPavlov - spelling corrector на основе метрики Левенштейна

In [ ]:
import os
from google.colab import drive
import zipfile
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import glob


drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
data_test = pd.read_csv('//content/gdrive/MyDrive/final_df_18_01.csv')

In [ ]:
!pip install deeppavlov
!pip install kenlm
!pip install sacremoses
from deeppavlov import build_model, configs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.4/489.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Successfully uninstalled tqdm-4.66.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully u

In [ ]:
model = build_model('levenshtein_corrector_ru', download=True)

def spelling_correction(col):
  return model([col])[0]



2024-01-18 09:51:40.838 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_data/vocabs/russian_words_vocab.dict.gz to /root/.deeppavlov/downloads/russian_words_vocab.dict.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/deeppavlov_data/vocabs/russian_words_vocab.dict.gz to /root/.deeppavlov/downloads/russian_words_vocab.dict.gz
100%|██████████| 10.7M/10.7M [00:01<00:00, 5.40MB/s]
2024-01-18 09:51:44.317 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting /root/.deeppavlov/downloads/russian_words_vocab.dict.gz archive into /root/.deeppavlov/downloads/vocabs
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/downloads/russian_words_vocab.dict.gz archive into /root/.deeppavlov/downloads/vocabs
2024-01-18 09:51:45.939 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/lang_models/ru_wiyalen_no_punkt.arpa.binary.gz to /roo

In [ ]:
data_test

,doc_name,doc_text
0,"Распоряжение, федеральный уровень, глава испол...",РАСПОРЯЖЕНИЕ ПРЕ...
1,"Распоряжение, федеральный уровень, глава испол...",РАСПОРЯЖЕНИЕ ПРЕ...
2,"Указ, федеральный уровень, глава исполнительно...",УКАЗ ПРЕ...
3,"Распоряжение, федеральный уровень, глава испол...",РАСПОРЯЖЕНИЕ ПРЕ...
4,"Распоряжение, федеральный уровень, глава испол...",РАСПОРЯЖЕНИЕ ПРЕ...
...,...,...
22886,"Указ, региональный уровень, глава исполнительн...",ГУБЕРНАТОР огловской ОБЛАСТИ У К АЗ ...
22887,"Указ, региональный уровень, глава исполнительн...",ммжхжчмитжкъщ. 3 а . . ...
22888,"Указ, региональный уровень, глава исполнительн...",У К А З ГУБЕРНАТОРА ПЕРМСКОГО КРАЯ 01.08.2...
22889,"Указ, региональный уровень, глава исполнительн...",У К А З ГУБЕРНАТОРА ПЕРМСКОГО КРАЯ 20.02.2...


In [ ]:
df_corrected = data_test.copy()
df_corrected['doc_text'] = df_corrected['doc_text'].apply(spelling_correction)
df_corrected

,doc_name,doc_text
0,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
1,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
2,"Указ, федеральный уровень, глава исполнительно...",указ президента российской федерации об освобо...
3,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
4,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
...,...,...
22886,"Указ, региональный уровень, глава исполнительн...",губернатор орловской области у к аз о мерах по...
22887,"Указ, региональный уровень, глава исполнительн...",ммжхжчмитжкъщ. 3 а.. 5 г х м 7 башкортостан ре...
22888,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края 01.08.2016 ...
22889,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края 20.02.2016 ...


In [ ]:
texts = df_corrected['doc_text'].tolist()
for i in range(10, 100, 10):
  print(texts[-i])

губернатора воронежской области с 1 октября 2131 га сна у г.воронеж об установлении ограничительных мероприятий карантина по бешенству животных на отдельной территории каменского муниципального района воронежской области в соответствии со статьей 17 закона российской федерации от 14.05.1993 49791 о ветеринарии на основании представления руководителя управления ветеринарии воронежской области вершина с.с. от 17.10.2017 63 01 1178 об установлении ограничительных мероприятий карантина по бешенству животных в целях предупреждения распространения заболевания животных бешенством на территории воронежской области постановляю 1. установить ограничительные мероприятия карантин до 20 декабря 2017 года включительно в неблагополучном по бешенству пункте хуторе ясеново каменского муниципального района воронежской области. 2. определить эпизоотическим очагом по бешенству животных индивидуальное подворье расположенное по адресу ул октябрьская д. 4 х ясеново каменский муниципальный район воронежская о

In [ ]:
df_corrected.to_csv('//content/gdrive/MyDrive/df_spell_corrected_18_01.csv', index=False)

In [ ]:
df_corrected

,doc_name,doc_text
0,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
1,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
2,"Указ, федеральный уровень, глава исполнительно...",указ президента российской федерации об освобо...
3,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
4,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
...,...,...
22886,"Указ, региональный уровень, глава исполнительн...",губернатор орловской области у к аз о мерах по...
22887,"Указ, региональный уровень, глава исполнительн...",ммжхжчмитжкъщ. 3 а.. 5 г х м 7 башкортостан ре...
22888,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края 01.08.2016 ...
22889,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края 20.02.2016 ...


In [ ]:
df_cor = pd.read_csv('//content/gdrive/MyDrive/df_spell_corrected_18_01.csv')

In [ ]:
import regex as re

In [ ]:
df_cor

,doc_name,doc_text
0,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
1,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
2,"Указ, федеральный уровень, глава исполнительно...",указ президента российской федерации об освобо...
3,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
4,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
...,...,...
22886,"Указ, региональный уровень, глава исполнительн...",губернатор орловской области у к аз о мерах по...
22887,"Указ, региональный уровень, глава исполнительн...",ммжхжчмитжкъщ. 3 а.. 5 г х м 7 башкортостан ре...
22888,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края 01.08.2016 ...
22889,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края 20.02.2016 ...


In [ ]:
def remove_numbers(string):
  numb = re.sub(r"\d+", "", string, flags=re.UNICODE)

  return re.sub(r'[^\w\s]', '', numb)


data_test = df_cor.copy()
data_test['doc_text'] = data_test['doc_text'].apply(remove_numbers)
data_test['doc_text'] = data_test['doc_text'].str.replace('\n', ' ')
data_test['doc_text'] = data_test['doc_text'].str.replace('-', ' ')
data_test

,doc_name,doc_text
0,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
1,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
2,"Указ, федеральный уровень, глава исполнительно...",указ президента российской федерации об освобо...
3,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
4,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...
...,...,...
22886,"Указ, региональный уровень, глава исполнительн...",губернатор орловской области у к аз о мерах по...
22887,"Указ, региональный уровень, глава исполнительн...",ммжхжчмитжкъщ а г х м башкортостан республи...
22888,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края об отмене...
22889,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края г об утве...


In [ ]:
data_test.to_csv('//content/gdrive/MyDrive/df_spell_corrected_22_01.csv', index=False)